In [2]:
import gradio as gr
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os
from fpdf import FPDF

# Cargar el modelo
model = tf.keras.models.load_model('modelo_clasificacion_basuraUltimateN.h5')

# Detalles de cada clase con más información sobre reciclaje y enlaces a videos de YouTube
class_details = {
    'Aluminium': {
        'es': 'Aluminio',
        'message': 'El aluminio es un material reciclable que se puede reutilizar sin perder calidad. Es utilizado en latas, papel de aluminio y otros productos.',
        'recycling_steps': '1. Limpia el aluminio.\n2. No lo mezcles con otros materiales.\n3. Llévalo a un centro de reciclaje.',
        'youtube_link': 'https://www.youtube.com/watch?v=5uGGBJtpvHk'
    },
    'Carton': {
        'es': 'Cartón',
        'message': 'El cartón reciclado se usa para hacer nuevos productos como cajas o papel. Se debe evitar que esté sucio o húmedo.',
        'recycling_steps': '1. Asegúrate de que el cartón esté limpio y seco.\n2. Dóblalo para ahorrar espacio.\n3. Recíclalo en un contenedor de papel.',
        'youtube_link': 'https://www.youtube.com/watch?v=ogU5DBZjJrQ'
    },
    'Glass': {
        'es': 'Vidrio',
        'message': 'El vidrio se puede reciclar sin perder calidad, como en botellas y frascos. Se debe evitar mezclarlo con otros materiales.',
        'recycling_steps': '1. Enjuaga bien las botellas y frascos.\n2. No mezcles vidrio de diferentes colores.\n3. Llévalo a un centro de reciclaje de vidrio.',
        'youtube_link': 'https://www.youtube.com/watch?v=Mi6yqch4Fas'
    },
    'Organic Waste': {
        'es': 'Residuos Orgánicos',
        'message': 'Restos de comida y jardín que se pueden compostar para hacer abono. Estos residuos son biodegradables.',
        'recycling_steps': '1. Separa los restos orgánicos de los residuos no reciclables.\n2. Haz compostaje en casa o llévalos a un centro especializado.',
        'youtube_link': 'https://www.youtube.com/watch?v=Fs8gT4Fhb6I'
    },
    'Other Plastics': {
        'es': 'Otros Plásticos',
        'message': 'Plásticos de un solo uso que pueden reciclarse, pero no todos. Asegúrate de verificar si son reciclables en tu área.',
        'recycling_steps': '1. Limpia los plásticos.\n2. Verifica si el plástico es reciclable en tu área.\n3. Llévalos a un centro de reciclaje especializado.',
        'youtube_link': 'https://www.youtube.com/watch?v=-nV3XfV3exM'
    },
    'Paper and Cardboard': {
        'es': 'Papel y Cartón',
        'message': 'Papel reciclado para hacer nuevos productos de papel. Asegúrate de que no tenga tinta o grasa.',
        'recycling_steps': '1. Separa el papel limpio del papel sucio o con grasa.\n2. Recíclalo en un contenedor de papel.',
        'youtube_link': 'https://www.youtube.com/watch?v=hgI4GFWpNvg'
    },
    'Plastic': {
        'es': 'Plástico',
        'message': 'Plástico reciclado para hacer envases y ropa, pero es importante reducir su uso. Algunos plásticos no son reciclables.',
        'recycling_steps': '1. Limpia el plástico.\n2. Verifica el tipo de plástico y su reciclabilidad.\n3. Recíclalo en el contenedor adecuado.',
        'youtube_link': 'https://www.youtube.com/watch?v=v8zwIqlTjQY'
    },
    'Textiles': {
        'es': 'Textiles',
        'message': 'Ropa reciclada que se puede convertir en nuevos productos o moda. La ropa en mal estado también puede ser reciclada.',
        'recycling_steps': '1. Separa la ropa reciclable.\n2. Llévala a un centro de reciclaje de textiles o dona ropa en buen estado.',
        'youtube_link': 'https://www.youtube.com/watch?v=0_c-xrjxzCI'
    },
    'Wood': {
        'es': 'Madera',
        'message': 'Madera reciclada para hacer muebles o energía. Es importante verificar si la madera tiene químicos antes de reciclarla.',
        'recycling_steps': '1. Verifica si la madera está libre de químicos.\n2. Llévala a un centro de reciclaje de madera.',
        'youtube_link': 'https://www.youtube.com/watch?v=CTgwmDkjqJk'
    }
}

# Función para hacer predicción
def predict_image(image_path):
    if not image_path:
        return "⚠️ Por favor, sube una imagen para clasificar.", None
    img = image.load_img(image_path)  # Cargar imagen
    img = img.resize((256, 256))  # Redimensionar imagen
    img_array = image.img_to_array(img)  # Convertir imagen a array
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalizar

    prediction = model.predict(img_array)  # Hacer predicción
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha
    confidence = np.max(prediction)  # Obtener la confianza de la predicción

    class_names = os.listdir(os.path.join(os.getcwd(), 'TrashDataset', 'train'))  

    predicted_class_name = class_names[predicted_class]
    predicted_class_es = class_details[predicted_class_name]['es']
    message = class_details[predicted_class_name]['message']
    recycling_steps = class_details[predicted_class_name]['recycling_steps']
    youtube_link = class_details[predicted_class_name]['youtube_link']

    # Crear PDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    colors = {  
        'Aluminium': (169, 169, 169), 
        'Carton': (255, 140, 0),  
        'Glass': (34, 139, 34),  
        'Organic Waste': (139, 69, 19),  
        'Other Plastics': (255, 69, 0),  
        'Paper and Cardboard': (30, 144, 255),  
        'Plastic': (255, 105, 180),  
        'Textiles': (255, 215, 0),  
        'Wood': (139, 69, 19)  
    }
    
    color = colors.get(predicted_class_name, (0, 0, 0))  
    pdf.set_text_color(*color)

    pdf.multi_cell(0, 10, f"Clasificación: {predicted_class_es}", align='C')
    pdf.multi_cell(0, 10, f"Confianza: {confidence * 100:.2f}%", align='C')
    pdf.multi_cell(0, 10, f"\n{message}", align='J')
    pdf.multi_cell(0, 10, f"\nPasos para reciclar:\n{recycling_steps}", align='J')
    pdf.multi_cell(0, 10, f"\nMás información en este video:\n{youtube_link}", align='J')

    pdf_output_path = f"{predicted_class_es}_informacion_reciclaje.pdf"
    pdf.output(pdf_output_path)

    return f'🌍 Clase: {predicted_class_es} con confianza de {confidence * 100:.2f}%', pdf_output_path

# Interfaz Gradio
def enhance_interface():
    interface = gr.Interface(fn=predict_image, 
                             inputs=gr.Image(type="filepath", label="📸 Carga una imagen de basura"), 
                             outputs=[gr.Textbox(label="🏆 Resultado de la Clasificación"), 
                                      gr.File(label="📄 Descargar PDF con detalles")],
                             live=True,
                             theme="default",
                             title="♻️ Clasificación de Residuos", 
                             description="🌱 Suba una imagen de basura y el modelo te dirá el tipo de basura y un archivo PDF con más detalles sobre cómo reciclarlo.",
                             submit_btn="Clasificar 🌱", 
                             clear_btn="Limpiar 🧹")

    return interface

# Crear y lanzar la interfaz
interface = enhance_interface()
interface.launch()

C:\Users\KEVIN\Downloads\ProjectF\projectovenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
